In [ ]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj

from yf_utils import _2_split_train_val_test, _3_random_slices, _4_perf_ranks
# from yf_utils import _2_split_train_val_test, _3_random_slices
# from trash import _4_perf_ranks_1

verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [ ]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)

In [ ]:
df_train, df_val, df_test = _2_split_train_val_test(df_c)

In [ ]:
len(df_train)
# df_val
# df_test

In [ ]:
n_samples = 1
days_lookback = 60
days_eval = 10

In [ ]:
my_slices = _3_random_slices(len(df_train), n_samples=n_samples, days_lookback=days_lookback, days_eval=days_eval)

In [ ]:
print(my_slices[0], my_slices[0][0], my_slices[0][1], my_slices[0][2])

In [ ]:
my_slices

In [ ]:
# for my_slice in my_slices[0:2]:
for my_slice in my_slices:
  start_train = my_slice[0]
  end_train = my_slice[1]
  start_val = end_train
  end_val = my_slice[2]
  print(f'start_train: {start_train}')
  print(f'end_train: {end_train}')
  print(f'start_val: {start_val}')
  print(f'end_val: {end_val}')

  # _df = df_train.iloc[start_train:end_train].copy()
  _df = df_train.iloc[start_train:end_train]
  print(_df) 
  # perf_ranks_dict, ranked_perf_ranks_dict = _4_perf_ranks(_df, days_lookbacks=[60])
  # print(perf_ranks_dict)
  # print(ranked_perf_ranks_dict)  

In [ ]:
perf_ranks_dict, ranked_perf_ranks_dict = _4_perf_ranks(_df, days_lookbacks=[60, 120])

In [ ]:
# print(perf_ranks_dict)
trash = perf_ranks_dict
trash

In [ ]:
ranked_perf_ranks_dict